<a href="https://colab.research.google.com/github/kryptohunterji/gianet/blob/main/TALLYXML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Excel to Tally XML - Google Colab GUI Version

import io
import pandas as pd
import xml.etree.ElementTree as ET

import ipywidgets as widgets
from IPython.display import display, clear_output

from google.colab import files

# Global storage for the uploaded DataFrame
state = {
    "df": None,
    "filename": None
}

# Widgets
upload_widget = widgets.FileUpload(
    accept=".xlsx",
    multiple=False,
    description="Upload Excel"
)

bank_ledger_widget = widgets.Text(
    description="Bank Ledger:",
    placeholder="Enter Bank Ledger Name"
)

suspense_ledger_widget = widgets.Text(
    description="Suspense:",
    placeholder="Enter Suspense/Party Ledger Name"
)

date_col_dropdown = widgets.Dropdown(
    description="Date Col:",
    options=[],
    disabled=True
)

narr_col_dropdown = widgets.Dropdown(
    description="Narration:",
    options=[],
    disabled=True
)

debit_col_dropdown = widgets.Dropdown(
    description="Debit Col:",
    options=[],
    disabled=True
)

credit_col_dropdown = widgets.Dropdown(
    description="Credit Col:",
    options=[],
    disabled=True
)

convert_button = widgets.Button(
    description="Convert to Tally XML",
    button_style="success"
)

status_output = widgets.Output()


def on_file_upload_change(change):
    with status_output:
        clear_output()
        if len(upload_widget.value) == 0:
            print("No file uploaded.")
            return

        # Get uploaded file
        upload_data = list(upload_widget.value.values())[0]
        content = upload_data["content"]
        filename = upload_data["metadata"]["name"]

        try:
            excel_bytes = io.BytesIO(content)
            df = pd.read_excel(excel_bytes)

            state["df"] = df
            state["filename"] = filename

            headers = df.columns.tolist()

            # Enable and populate dropdowns
            for dd in [date_col_dropdown, narr_col_dropdown, debit_col_dropdown, credit_col_dropdown]:
                dd.options = headers
                dd.disabled = False

            print(f"File '{filename}' uploaded successfully.")
            print("Columns detected:", headers)

        except Exception as e:
            print("Error reading Excel file:", e)


def convert_to_tally_xml(button):
    with status_output:
        clear_output()

        df = state.get("df")
        filename = state.get("filename")

        if df is None:
            print("Please upload an Excel file first.")
            return

        bank_ledger = bank_ledger_widget.value.strip()
        suspense_ledger = suspense_ledger_widget.value.strip()

        if not bank_ledger or not suspense_ledger:
            print("Please fill both Bank Ledger and Suspense/Party Ledger names.")
            return

        date_col = date_col_dropdown.value
        narr_col = narr_col_dropdown.value
        debit_col = debit_col_dropdown.value
        credit_col = credit_col_dropdown.value

        if not all([date_col, narr_col, debit_col, credit_col]):
            print("Please select all column mappings.")
            return

        try:
            # Build XML structure
            envelope = ET.Element("ENVELOPE")
            header = ET.SubElement(envelope, "HEADER")
            tally_req = ET.SubElement(header, "TALLYREQUEST")
            tally_req.text = "Import Data"

            body = ET.SubElement(envelope, "BODY")
            import_data = ET.SubElement(body, "IMPORTDATA")

            request_desc = ET.SubElement(import_data, "REQUESTDESC")
            report_name = ET.SubElement(request_desc, "REPORTNAME")
            report_name.text = "All Masters"

            request_data = ET.SubElement(import_data, "REQUESTDATA")

            for index, row in df.iterrows():
                # Date format YYYYMMDD
                try:
                    date_value = pd.to_datetime(row[date_col]).strftime("%Y%m%d")
                except Exception:
                    # If date is invalid, skip row
                    continue

                narration = str(row[narr_col])

                try:
                    debit = float(row[debit_col]) if pd.notna(row[debit_col]) else 0
                except Exception:
                    debit = 0

                try:
                    credit = float(row[credit_col]) if pd.notna(row[credit_col]) else 0
                except Exception:
                    credit = 0

                # Skip if both are zero or NaN
                if credit <= 0 and debit <= 0:
                    continue

                # Logic: classify as Receipt or Payment
                if credit > 0:
                    voucher_type = "Receipt"
                    bank_amount = -abs(credit)
                    suspense_amount = abs(credit)
                    bank_pos = "Yes"
                    suspense_pos = "No"
                else:
                    voucher_type = "Payment"
                    bank_amount = abs(debit)
                    suspense_amount = -abs(debit)
                    bank_pos = "No"
                    suspense_pos = "Yes"

                # TALLYMESSAGE + VOUCHER
                tally_msg = ET.SubElement(request_data, "TALLYMESSAGE")
                voucher = ET.SubElement(
                    tally_msg,
                    "VOUCHER",
                    {
                        "VCHTYPE": voucher_type,
                        "ACTION": "Create",
                        "OBJVIEW": "Accounting Voucher View",
                    },
                )

                ET.SubElement(voucher, "DATE").text = date_value
                ET.SubElement(voucher, "VOUCHERTYPENAME").text = voucher_type
                ET.SubElement(voucher, "NARRATION").text = narration
                ET.SubElement(voucher, "PARTYLEDGERNAME").text = bank_ledger
                ET.SubElement(voucher, "EFFECTIVEDATE").text = date_value

                # Suspense ledger entry
                entry1 = ET.SubElement(voucher, "ALLLEDGERENTRIES.LIST")
                ET.SubElement(entry1, "LEDGERNAME").text = suspense_ledger
                ET.SubElement(entry1, "ISDEEMEDPOSITIVE").text = suspense_pos
                ET.SubElement(entry1, "AMOUNT").text = f"{suspense_amount:.2f}"

                # Bank ledger entry
                entry2 = ET.SubElement(voucher, "ALLLEDGERENTRIES.LIST")
                ET.SubElement(entry2, "LEDGERNAME").text = bank_ledger
                ET.SubElement(entry2, "ISDEEMEDPOSITIVE").text = bank_pos
                ET.SubElement(entry2, "AMOUNT").text = f"{bank_amount:.2f}"

                bank_alloc = ET.SubElement(entry2, "BANKALLOCATIONS.LIST")
                ET.SubElement(bank_alloc, "DATE").text = date_value
                ET.SubElement(bank_alloc, "TRANSACTIONTYPE").text = "Cheque"
                ET.SubElement(bank_alloc, "PAYMENTFAVOURING").text = suspense_ledger
                ET.SubElement(bank_alloc, "AMOUNT").text = f"{bank_amount:.2f}"

            # Output file name
            base_name = filename.rsplit(".", 1)[0] if filename else "output"
            output_path = f"{base_name}_Tally.xml"

            tree = ET.ElementTree(envelope)
            tree.write(output_path, encoding="utf-8", xml_declaration=True)

            print(f"XML generated successfully: {output_path}")
            print("Starting download...")
            files.download(output_path)

        except Exception as e:
            print("Error during conversion:", e)


# Wire up events
upload_widget.observe(on_file_upload_change, names="value")
convert_button.on_click(convert_to_tally_xml)

# Layout
ui = widgets.VBox([
    widgets.HTML("<h3>Excel to Tally XML Converter (Google Colab)</h3>"),
    upload_widget,
    bank_ledger_widget,
    suspense_ledger_widget,
    date_col_dropdown,
    narr_col_dropdown,
    debit_col_dropdown,
    credit_col_dropdown,
    convert_button,
    status_output
])

display(ui)
